In [1]:
import torch

torch.cuda.is_available()
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment
from capstone import *
from capstone.x86 import *

import os
import json 

import torch
from torch.utils.data import DataLoader, TensorDataset
import sys,os
from sklearn.metrics import precision_recall_fscore_support , accuracy_score
import numpy as np

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'


In [2]:

MAX_TOKEN_LEN = 1024
BATCH_SIZE = 40
epochs = 100

EXPERIMENT_NAME = 'cusTokenizer_bartOwn_35k_ATW_signature'

# disassembly_decimal disassembly_all_number_to_words disassembly_decimal 
data_key = "disassembly_all_number_to_words"


from transformers import BartTokenizer,BertTokenizer, BertForNextSentencePrediction,BertForPreTraining,BertConfig,AutoModelForMaskedLM,get_linear_schedule_with_warmup ,BertForSequenceClassification
import torch
from transformers import PreTrainedTokenizerFast ,AutoModelForPreTraining ,BertGenerationEncoder, BartForConditionalGeneration,BertGenerationDecoder,EncoderDecoderModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
tokenizer = PreTrainedTokenizerFast.from_pretrained("./../models/bart_AT" )
print(tokenizer.pad_token) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


<pad>


In [3]:
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
cls_token = tokenizer.cls_token

In [4]:
MAX_TOKEN_LEN = 1024



# disassembly_decimal disassembly_all_number_to_words disassembly_decimal  disassembly_addresses_to_words
data_key = "disassembly_decimal"





# def generate_param_string(param_list):


#     if len(param_list)==0:
#         return 'void'
#     else:
#         return " ".join(param_list)
def unique_items(list_of_lists):
    # Flatten the list of lists
    print(list_of_lists[3].split())
    flattened_list = [item for sublist in list_of_lists for item in sublist.split()]
    # Convert the flattened list to a set to remove duplicates
    unique_elements = set(flattened_list)
    return list(unique_elements)






DATA_PATH = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions'

TRAIN_DATA_PATH  ='/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/train/'

TEST_DATA_PATH   = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/test/'




train_json_files = [os.path.join(TRAIN_DATA_PATH, f) for f in os.listdir(TRAIN_DATA_PATH) ]

test_json_files = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) ]




all_types = ['*short', '*ulong', 'int', 'ulonglong', '*longlong', 'None', 'short', 'char', '*struct', '*long', '*char', 'double', 'uint', 'uchar', '*float', 'longlong', 'long', '*void', 'float', 'bool', '*int', '*ushort', '*ulonglong', '*uchar', 'ushort', 'void', '*double', 'ulong', '*uint', '*bool']
all_types_dict = {}
for k,type in enumerate(all_types):
    all_types_dict[type] = str(k)

def read_corpus(json_files):

    all = []
    all_params = []
    all_returns = []
    all_signatures = []

    for k, j_file in enumerate(json_files):
        # if k>20000:
        #     break
        try:

            with open(j_file, 'r') as file:
                data = json.load(file)

                funct = data[data_key]['input']
                
                all.append(funct)

                param_string = ""

                if len(data['signature']['paramTypes'])==0:
                    param_string = all_types_dict['void']
                    
                for v,type in enumerate(data['signature']['paramTypes']):
                    if v==0:
                        param_string =  all_types_dict[type]
                    else:
                        param_string = param_string + sep_token +all_types_dict[type]
                
                all_params.append( param_string )
                all_returns.append(  data['signature']['returnType']  )
                all_signatures.append(param_string + sep_token + all_types_dict[data['signature']['returnType']] )
        except Exception as e: 
            print(e)
    return all , all_params , all_returns , all_signatures
    


train_text , train_params , train_returns, train_signatures = read_corpus(train_json_files)

test_text  , test_params  , test_returns , test_signatures = read_corpus(test_json_files)



        
# text = text[0:5000]
print("Functions Count: ",len(train_text), '\n')
example = train_text[10]
text = train_text + test_text
signatures = train_signatures + test_signatures
input_output_data = []
for i in range(len(text)):
    input_output_data.append(text[i])
    input_output_data.append(signatures[i])
    
unique_types = unique_items(train_params)
print(len(unique_types))




Functions Count:  80000 

['2</s>8']
1566


In [5]:
def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=1024, return_tensors="pt")

train_source_encodings = tokenize(train_text)
train_target_encodings = tokenize(train_signatures)


test_source_encodings = tokenize(test_text)
test_target_encodings = tokenize(test_signatures)

# Create a TensorDataset and DataLoader
train_dataset = TensorDataset(train_source_encodings['input_ids'], train_source_encodings['attention_mask'], train_target_encodings['input_ids'])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


test_dataset = TensorDataset(test_source_encodings['input_ids'], test_source_encodings['attention_mask'], test_target_encodings['input_ids'])
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)




In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base", forced_bos_token_id=0)
# tok = BartTokenizer.from_pretrained("facebook/bart-large")
# example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
# batch = tok(example_english_phrase, return_tensors="pt")
# generated_ids = model.generate(batch["input_ids"])
# assert tok.batch_decode(generated_ids, skip_special_tokens=True) == [
#     "UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria"
# ]

In [7]:
# from transformers import LongformerModel, LongformerTokenizer

# # Load tokenizer and model
# tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
# model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

# # Example of how to prepare inputs
# inputs = tokenizer("Example of a long input sequence " * 50, return_tensors="pt", max_length=1024, truncation=True)
# output = model(**inputs)

In [8]:
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
cls_token_id = tokenizer.cls_token_id



# def align_output(pred, gt):
#     params_pred = []
#     params_gt = []
#     ret_pred = []
#     ret_gt = []
#     for i,gid in enumerate(gt):
#         if gid!= cls_token_id:
            
            
#             aligned_pred.append(pred[i])
#             aligned_gt.append(gid)

            

In [9]:

model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [10]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)





# Training loop



for epoch in range(epochs):  # Number of epochs
    model.train()
    all_prediction = []
    all_ground_truth=[]
    for n,batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        # print(input_ids.shape , attention_mask.shape , labels.shape )
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        token_predictions = torch.argmax(outputs.logits, axis=-1)


                    
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        labels_list = labels.tolist()
        token_predictions_list = token_predictions.tolist()
        
        for  j,label in enumerate(labels_list):
            for i, lid in enumerate(label):
                if lid not in  [cls_token_id , sep_token_id , pad_token_id]:

                    all_ground_truth.append(lid)
                    all_prediction.append(token_predictions_list[j][i])
                            

        if n%200==0 and n!=0:
            # print(f"Epoch {epoch}, Loss: {loss.item()}")
            
            # print(labels[0].tolist())
            # print(token_predictions[0])

            
            pred_tokens = tokenizer.convert_ids_to_tokens(token_predictions[0].tolist())
            gt_tokens = tokenizer.convert_ids_to_tokens(labels[0].tolist())
        
            print("pred_tokens: ",pred_tokens)
            print("gt_tokens: ",gt_tokens)
        

            print ('epoch: ',epoch , ' n: ', n , ' loss ',loss.item())
            print(len(all_ground_truth) , len(all_prediction))
            print(all_ground_truth[-10:])
            print(all_prediction[-10:])
            precision, recall, f1, _ = precision_recall_fscore_support(all_ground_truth,all_prediction ,average='weighted')
            print('Train precision, recall, f1' ,precision, recall, f1)
        
            print(labels_list[0])
            print(token_predictions_list[0])
    

    # model.save_pretrained("./../models/"+EXPERIMENT_NAME+"_model.ckpt")
    # print('saved')
    # continue
##########################################
    #eval
    model.eval()
    with torch.no_grad():
        test_all_ground_truth = []
        test_all_prediction = []
        right = 0
        wrong = 0
        for n,test_batch in enumerate(test_loader):
    
            test_input_ids, test_attention_mask, test_labels = [x.to(device) for x in test_batch]
            # print(input_ids.shape , attention_mask.shape , labels.shape )
            test_outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask, labels=test_labels)
            test_token_predictions = torch.argmax(test_outputs.logits, axis=-1)
    
            test_labels_list = test_labels.tolist()
            test_token_predictions_list = test_token_predictions.tolist()
            
            for  j,test_label in enumerate(test_labels_list):
                if test_label==test_token_predictions_list[j]:
                    right = right +1
                else:
                    wrong = wrong+ 1
                for i, lid in enumerate(test_label):
                    if lid not in  [cls_token_id , sep_token_id , pad_token_id]:
                        test_all_ground_truth.append(lid)
                        test_all_prediction.append(test_token_predictions_list[j][i])
    
    
    
    
        print("# # #" *10)
        # test_pred_tokens = tokenizer.convert_ids_to_tokens(test_token_predictions[0].tolist())
        # test_gt_tokens = tokenizer.convert_ids_to_tokens(test_labels[0].tolist())
        # print('test_pred_tokens' , test_pred_tokens)
        # print('test_gt_tokens' ,test_gt_tokens)
        print ('epoch: ',epoch , ' n: ', n )
        print(len(test_all_ground_truth) , len(test_all_prediction))
        print(test_all_ground_truth[-10:])
        print(test_all_prediction[-10:])
        test_accuracy = (accuracy_score(test_all_prediction, test_all_ground_truth))
        test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_all_ground_truth,test_all_prediction ,average='weighted')
        print('TEST precision, recall, f1 ,  accuracy' ,test_precision, test_recall, test_f1 , test_accuracy)
    
        print(test_labels_list[0])
        print(test_token_predictions_list[0])
        print("right",right, "  wrong : ", wrong)
    
        # Save the model
    
        model.save_pretrained("./../models/"+EXPERIMENT_NAME+"_model.ckpt")

/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


pred_tokens:  ['<s>', '2', '</s>', '2', '</s>', '8', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
gt_tokens:  ['<s>', '8', '</s>', '2', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


pred_tokens:  ['<s>', '25', '</s>', '8', '</s>', '2', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
gt_tokens:  ['<s>', '2', '</s>', '8', '</s>', '2', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

pred_tokens:  ['<s>', '25', '</s>', '25', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
gt_tokens:  ['<s>', '25', '</s>', '25', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


pred_tokens:  ['<s>', '25', '</s>', '2', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
gt_tokens:  ['<s>', '25', '</s>', '2', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raisul/anaconda3/envs/pytorch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

In [ ]:
print(sep_token_id ,pad_token_id,cls_token_id)

In [ ]:
tokenizer('[', 'SEP', ']')